In [25]:
from groq import Groq
import json
import re
import glob
import os
API_KEY = "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"
#API_KEY = "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"

In [26]:
client = Groq (api_key=API_KEY)

In [11]:
def some_shots_qwen (sentence):   
    completion = client.chat.completions.create(
        model="qwen-2.5-32b",
        messages=[
            {
                "role": "user",
                "content": "An entity is : \
                 -a person or a collective of persons (PER) \
                     e.g. The Beatles, Gandhi \
                 - an organisation (ORG)\
                     e.g. Musée du Louvres, NBA \
                 - a location (LOC) \
                     e.g. China, Les Alpes \
                 - or miscellaneous (MISC), \
                    which can be events, brands, function, etc. \
                    e.g. World Chess Championship, Officier\
                Find the entities in the followning sentence : "+sentence+"\nTHe output should\
                be json in the format \{'entities':[\{'entity':text, 'type':type\},...]\}.\
                Determiners (le, la, ...) should not be included\n"
            }
        ],
        temperature=0.6,
        max_completion_tokens=4096,
        top_p=0.95,
        stream=True,
        stop=None,
    )
    result_raw = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content
        if content :
            result_raw += content
    #print(result_raw)
    
    #print("-"*44)
    json_raw = ""
    json_flag = False
    for line in result_raw.split("\n"):
        
        if line == "```" or line=="```json":
            json_flag = not json_flag
        elif json_flag: 
            json_raw+=line +"\n"
            
    #print(json_raw)
    try:
        entities =  json.loads (json_raw)
    except:
        entities = {}
    return entities

    
def add_entities_index (entities,sentence, offset):
    sentence_lower = sentence.lower()
    if 'entities' in entities:
        for entity in entities['entities']:
            
            text = entity["entity"]
            if text.lower() in sentence_lower:
                start = sentence_lower.index (text.lower())
            else:
                # There can be problems with some spaces : e.g. d' Actium in the original text
                # and d'Actium in the LLM output 
                splitted_text =  text.lower().replace("'"," ").split(" ")
                # VERY RUDIMENTARY SOLUTION HERE
                # TODO : CHANGE THIS
                if splitted_text[0] in sentence_lower and splitted_text[-1] in sentence_lower:
                    start = sentence_lower.index (splitted_text[0])
                else:
                    print(text, "not in", sentence)
                    continue
                
            end = start + len (text)
            entity["start"] = start + offset
            entity ["end"] = end + offset
            
                
                        

def entities_to_ann (entities):
    ann = ""
    for i, entity in enumerate(entities):
        if 'start' in entity and 'end' in entity:
            ann += f"T{i+1}\t{entity['type']} {entity['start']} {entity['end']}\t{entity['entity']}\n"
    return ann

    

<>:17: SyntaxWarning: invalid escape sequence '\{'
<>:17: SyntaxWarning: invalid escape sequence '\{'
/var/folders/qt/q5s0yg0959d5b540drdr9xpc0000gn/T/ipykernel_32209/3301045405.py:17: SyntaxWarning: invalid escape sequence '\{'
  Find the entities in the followning sentence : "+sentence+"\nTHe output should\


In [12]:

for file in glob.glob("FENEC/*.txt"):
    print(file)

    all_entities = []
    with open(file,"r") as f:
        offset = 0
        for line in f: 
            chunks = [line]
            if len(line)>=1000:
                # BRUTAL
                # TO CHANGE
                chunks = [line[1000*i:1000*(i+1)] for i in range(len(line)//1000)]
            for chunk in chunks:    
                entities = some_shots_qwen (chunk)
                add_entities_index (entities, chunk, offset)
                if 'entities' in entities:
                    all_entities += (entities['entities'])
                offset += len(chunk)
    ann_file = "systems-outputs/original-tagsets/qwen/"+os.path.basename(file).split(".")[0]+".ann"
    with open(ann_file, "w") as f:
        f.write (entities_to_ann (all_entities))

FENEC/encyclopedia01-WikiNER.txt
FENEC/multi03-FQB.txt
FENEC/prose03-Giono.txt
FENEC/information01-APIL.txt
mètre not in BERULLE - à 18 h profitez des conditions de l’Espace Jean Musy pour faire un passage sur scène : musique, poésie, danse, sketchs, acrobatie, etc. Une soirée unique chaque année ! Scène ouverte à tous sur inscription uniquement 03 25 42 59 34 – entrée libre, tout public.

FENEC/spoken01-Rhapsodie.txt
FENEC/multi02-Sequoia.txt
The Beatles not in Distribution :

Musée du Louvre not in Distribution :

Paris not in Distribution :

France not in Distribution :

FENEC/information02-Wikinews.txt
Russie not in La semaine dernière, le Royaume-Uni a déjà expulsé 23 diplomates russes.

groupe Bollé not in Le groupe Bolloré (Canal+) y voit une volonté de discrimination à l'encontre des téléspectateurs selon leur mode de réception.

Chine not in Espace : la station spatiale chinoise Tiangong-1 retombera sur Terre début avril  Page actualisée : 4 mars 2018 — « Espace : la station s

In [14]:

def more_shots_qwen (sentence):   
    completion = client.chat.completions.create(
        model="qwen-2.5-32b",
        messages=[
            {
                "role": "user",
                "content": "An entity is : \
                 -a person or a collective of persons (PER) \
                     e.g. The Beatles, Gandhi \
                 - an organisation (ORG)\
                     e.g. Musée du Louvres, NBA \
                 - a location (LOC) \
                     e.g. China, Les Alpes \
                 - or miscellaneous (MISC), \
                    which can be events, function, etc. \
                    e.g. World Chess Championship, Officier\
                \nPerform Named entity recognition on the given sentences. The output should\
                be json in the format \{'entities':[\{'entity':text, 'type':type\},...]\}.\
                Determiners (le, la, ...) should not be included\n"
            },
            {
                "role":"user",
                "content":"Le paysan toulousain René se baigne au bassin du Havre, la princesse Sisi reçoit le baiser de Poséidon vers Paris, se promenant au bord de la Seine."
            },
            {
                "role": "assistant",
                "content": """```json
                {
                  "entities": [
                    {"entity": "paysan", "type": "MISC"},
                    {"entity": "René", "type": "PER"},
                    {"entity": "bassin du Havre", "type": "LOC"},
                    {"entity": "princesse", "type": "MISC"}
                    {"entity": "Sisi", "type": "PER"},
                    {"entity": "Poséidon", "type": "PER"},
                    {"entity": "Paris", "type": "LOC"},
                    {"entity": "Seine", "type": "LOC"}
                  ]
                }
                ```"""
                
            },
            {
                "role":"user",
                "content":"Non seulement Haiyan est responsable d'une hausse de l'humidité mais aussi une canicule arrivera bientôt."
            },

           {
                "role":"assistant",
                "content":"""```json
                {
                  "entities": [
                    {"entity": "Haiyan", "type": "MISC"}
                  ]
                }
                ```"""
            },
            {
                "role":"user",
                "content": sentence
            },
            
            
            
        ],
        temperature=0.6,
        max_completion_tokens=4096,
        top_p=0.95,
        stream=True,
        stop=None,
    )
    result_raw = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content
        if content :
            result_raw += content
    json_raw = ""
    json_flag = False
    for line in result_raw.split("\n"):
        
        if line == "```" or line=="```json":
            json_flag = not json_flag
            if not json_flag:
                break
        elif json_flag:
            json_raw+=line +"\n"
            
    try:
        entities =  json.loads (json_raw)
    except:
        
        entities = {}
    return entities

more_shots_qwen ("Lesbos où les Phrynés l'une l'autre s'attirent, Où jamais un soupir ne resta sans écho, A l'égal de Paphos les étoiles t'admirent, Et Vénus à bon droit peut jalouser Sapho! --Lesbos où les Phrynés l'une l'autre s'attirent.")

<>:7: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\{'
/var/folders/qt/q5s0yg0959d5b540drdr9xpc0000gn/T/ipykernel_32209/143297168.py:7: SyntaxWarning: invalid escape sequence '\{'
  "content": "An entity is : \


{'entities': [{'entity': 'Lesbos', 'type': 'LOC'},
  {'entity': 'Phrynés', 'type': 'PER'},
  {'entity': 'Paphos', 'type': 'LOC'},
  {'entity': 'Vénus', 'type': 'PER'},
  {'entity': 'Sapho', 'type': 'PER'}]}

In [30]:
for file in sorted(glob.glob("FENEC/*.txt"))[11:]: 
    print(file)
    all_entities = []
    with open(file,"r") as f:
        offset = 0
        for line in f: 
            chunks = [line]
            if len(line)>=1000:
                # BRUTAL
                # TO CHANGE
                chunks = [line[1000*i:1000*(i+1)] for i in range(len(line)//1000)]
            for chunk in chunks:    
                entities = more_shots_qwen (chunk)
                add_entities_index (entities, chunk, offset)
                if 'entities' in entities:
                    
                    all_entities += (entities['entities'])
                offset += len(chunk)
    ann_file = "systems-outputs/original-tagsets/qwen_more_shots/"+os.path.basename(file).split(".")[0]+".ann"
    with open(ann_file, "w") as f:
        f.write (entities_to_ann (all_entities))

FENEC/prose03-Giono.txt
FENEC/spoken01-Rhapsodie.txt
FENEC/spoken02-Rhapsodie.txt
outre-mer not in si poïésis a donné le mot poésie euh aisthesis a donné le terme esthétique à chacune et à chacun d' entre vous Françaises et Français de métropole d' outre - mer de l' étranger je souhaite très chaleureusement une bonne et une heureuse année deux mille il centre Agüero pour qui Rodriguez Rodriguez Rodriguez et les Argentins sous les ordres de Maradona euh jouaient un peu plus haut mh donc euh on voit Chaplin euh s' habiller pour sortir de son usine puisque les les les les c~ les capitales les grandes villes ne me disaient rien du tout bah euh oh bah de l' arrêt l' arrêt de Steve Mandanda à la suite de ce ballon arraché euh par les euh Argentins à la défense de cette équipe de France non mais même pas dans le fond je suis très très pudique et c' est vrai que bon je les vois pas partir le matin alors là vous faisiez des études de médecine déjà il y a un moment euh ils disaient que de toute 